<a href="https://colab.research.google.com/github/ALOK158/AI_Text_Detector/blob/main/GPT_FROM_SCRATCH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import torch
import pandas as pd
import numpy as np

In [ ]:
#Lets start with our data set that is of tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-06-28 08:38:10--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.009s  

2025-06-28 08:38:10 (113 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
# Lets read the text
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [ ]:
print("Length of the dataset in characters: ", len(text))


Length of the dataset in characters:  1115394


In [ ]:
#First 1000 char
print(text[:10000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [ ]:
# Get all uniuqe char in the text
chars=sorted(list(set(text)))
vocab_size=len(chars)
print(''.join(chars))
print(len(chars))



 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


##Tokenizing

Simplest way of tokenization- char level tokenization

In future we will explore more in this for to improve our model

In [ ]:
stoi={ch:i for i,ch in enumerate(chars)}
itos={i:ch for i,ch in enumerate(chars)}
encode=lambda s:[stoi[c] for c in s]
decode=lambda l:''.join([itos[i] for i in l])


In [ ]:
# lets encode our Shakespeare Text
data=torch.tensor(encode(text),dtype=torch.long)
print(data.shape, data.type)

torch.Size([1115394]) <built-in method type of Tensor object at 0x788c81711fd0>


Lets Split the data in Traing and Validation

In [ ]:
n=int(0.9*len(data))
train_data=data[:n]
val_data=data[n:]

In [ ]:
block_size=8# sparese data in 8 times

In [ ]:
x= train_data[:block_size]
y=train_data[1:block_size+1]
for t in range(block_size):
  context=x[:t]
  target=y[t]
  print(f"when input is {context} the target is {target}")

when input is tensor([], dtype=torch.int64) the target is 47
when input is tensor([18]) the target is 56
when input is tensor([18, 47]) the target is 57
when input is tensor([18, 47, 56]) the target is 58
when input is tensor([18, 47, 56, 57]) the target is 1
when input is tensor([18, 47, 56, 57, 58]) the target is 15
when input is tensor([18, 47, 56, 57, 58,  1]) the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 58


splitting our data in smaller chunks

In [ ]:
torch.manual_seed(1337)
batch_size=4# Independent sequnece gets proccess in pararell
block_size=8 # Max context length

def get_batch(split):
  data=train_data if split=='train' else val_data
  ix=torch.randint(len(data)- block_size, (batch_size,))
  x=torch.stack([data[i:i+block_size] for i in ix])
  y=torch.stack([data[i+1:i+1+block_size] for i in ix])
  return x, y

xb, yb= get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----


BASELINE MODEL

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) # (B,T,C)
        if targets is None:
            loss = None
        else:
          B, T, C=logits.shape
          logits=logits.view(B*T,C)
          targets=targets.view(B*T)
          loss=F.cross_entropy(logits, targets)
        return logits, loss
    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in current context
        for _ in range(max_new_tokens):
          #do the prediciton
          logits, loss=self(idx)
          #focus only on last time step
          logits=logits[:,-1,:]
          #apply softmax to do prediction
          probs=F.softmax(logits, dim=-1)# (B,C)
          #sample from distribution
          idx_next=torch.multinomial(probs, num_samples=1)#(B,1)
          #append sample index to running sequnece
          idx=torch.cat((idx, idx_next), dim=1)#(B,T+1)
        return idx



m= BigramLanguageModel(vocab_size)
logits, loss=m(xb,yb)

print(loss)

idx=torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx=torch.zeros((1,1), dtype =torch.long), max_new_tokens=100)[0].tolist()))

tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


Lets train the model

In [ ]:
from torch.optim import AdamW
# Create pytorch optimizer
optimizer=torch.optim=AdamW(m.parameters(), lr=1e-4)


In [ ]:
batch_size=16
for steps in range(10000):
  #sample a batch of data
  xb,yb=get_batch('train')
  #evaluate the loss
  logits, loss=m(xb,yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

3.753307342529297


In [ ]:
print(decode(m.generate(idx=torch.zeros((1,1), dtype =torch.long), max_new_tokens=100)[0].tolist()))


vv.DWgzta!jKdaP!jXXEZKT:zz;:ZycTI&Q?Ezma;AUNKl!
phTkV& !!$gCprIxly$gO'd,!jHDYdmOL:xfuD&VBjR
Ju;h'qoT


Maths Trick in self-attention  /

For processing current token we will  need to carry info from all the previous tokens- one way - the average of all precious tokens

In [ ]:
torch.manual_seed(1337)
B,T,C=4,8,2 # btach, time, channel
x= torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

How to do cal efficinetly??


In [ ]:
#Version 1
wei=torch.tril(torch.ones(T,T))
wei=wei/wei.sum(1, keepdim=True)
xbow2=wei @ x## (T,T)@ (B,T,C)----Pytorch---(B,T,T) @(B,T,C)
xbow2

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])

In [ ]:
t=torch.tril(torch.ones(T,T))

In [ ]:
#ver 2
torch.manual_seed(1337)
B,T,C=4,8,2 # btach, time, channel
x= torch.randn(B,T,C)

# head of self-Attention
head_size=16
key=nn.Linear(C, head_size, bias=False)
query=nn.Linear(C, head_size, bias=False)
value=nn.Linear(C, head_size, bias=False)

k=key(x) #(B,T,16)
q=query(x) #(B,T,16)
wei=q @ k.transpose(-2,-1) # (B,T,16) @ (B,T,16) --(B,T,16) @ (B,16,T)-> (B,T,T)



t=torch.tril(torch.ones(T,T))
wei=torch.zeros((T,T))
wei=wei.masked_fill(t==0, float('-inf'))
wei=F.softmax(wei, dim=1)
wei
# wei get the same as we get previously
# -- previous info and current get togehter --- avergae out

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [ ]:
#ver 2
torch.manual_seed(1337)
B,T,C=4,8,32 # btach, time, channel
x= torch.randn(B,T,C)

# head of self-Attention---> So evey token have 2 values--- @Query(what it looking) & @ Key(what it has)
head_size=16
key=nn.Linear(C, head_size, bias=False)
query=nn.Linear(C, head_size, bias=False)
value=nn.Linear(C, head_size, bias=False)

k=key(x) #(B,T,16)
q=query(x) #(B,T,16)
wei=q @ k.transpose(-2,-1) # (B,T,16) @ (B,T,16) --(B,T,16) @ (B,16,T)-> (B,T,T)



t=torch.tril(torch.ones(T,T))
wei=wei.masked_fill(t==0, float('-inf'))
wei=F.softmax(wei, dim=-1)
v=value(x)
out =wei @v
# wei get the same as we get previously
# -- previous info and current get togehter --- avergae out

In [ ]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)
# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 1e—3

eval_iters = 200
n_embd = 32


SyntaxError: invalid decimal literal (ipython-input-13-3009318968.py, line 10)

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)
# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3 # Corrected hyphen to minus sign
dropout = 0.1 # Added dropout definition

eval_iters = 200
n_embd = 32

class Head(nn.Module):
  """Single Head of Self Attention"""
  def __init__(self, head_size):
    super().__init__()
    self.key=nn.Linear(n_embd, head_size, bias=False)
    self.query=nn.Linear(n_embd, head_size, bias=False)
    self.value=nn.Linear(n_embd, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
  def forward(self,x):
    B,T,C=x.shape
    k=self.key(x)
    q=self.query(x)
    #Compute attention score(""affinities)
    wei =q @ k.transpose(-2,-1)* C**-0.5
    wei= wei.masked_fill(self.tril[:T,:T]==0, float('-inf')) # Corrected trill to tril
    wei=F.softmax(wei, dim=-1)
    #perfrom the weighted aggregation of values
    v=self.value(x)# (B,T,C) # Corrected values to value
    out =wei @ v
    return out

class FeedForward(nn.Module):
  """A simple linear layer followed by a non-linearity"""
  def __init__(self, n_embd):
    super().__init__()
    self.net=nn.Sequential(
        nn.Linear(n_embd, n_embd),
        nn.ReLU(),
      )
  def forward(self, x):
     return self.net(x)




class MultiHeadAttention(nn.Module):
  """Multiple heads of self attention model----By rrunning multiple heads in parrallel"""
  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads=nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj=nn.Linear(n_embd, n_embd)
    self.dropout=nn.Dropout(dropout)
  def forward(self, x):
    return torch.cat([h(x) for h in self.heads], dim=-1)
    self.proj(x)
    self.dropout(x)

class BigramLanguageModel(nn.Module): # Corrected indentation
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd) # Added positional embedding
        self.sa_head= MultiHeadAttention(4, n_embd//4)# 4 head of 8-dimenional self-attention
        self.ffwd=FeedForward(n_embd)
        self.lm_head=nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape # Get B and T from idx
        #positional encoding
        tok_embd = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device)) # (T,C), use idx.device
        x = tok_embd + pos_emb # (B,T,C)
        x=self.sa_head(x)#Applied one self attention head
        x=self.ffwd(x)
        logits = self.lm_head(x) # (B,T,vocab_size)
        if targets is None:
            loss = None
        else:
          B, T, C=logits.shape
          logits=logits.view(B*T,C)
          targets=targets.view(B*T)
          loss=F.cross_entropy(logits, targets)
        return logits, loss
    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in current context
        for _ in range(max_new_tokens):
          # crop idsx to the last bloc_size tokens
          idx_cond=idx[:, -block_size:]
          #do the prediciton
          logits, loss=self(idx_cond) # Use idx_cond for prediction
          #focus only on last time step
          logits=logits[:,-1,:]
          #apply softmax to do prediction
          probs=F.softmax(logits, dim=-1)# (B,C)
          #sample from distribution
          idx_next=torch.multinomial(probs, num_samples=1)#(B,1)
          #append sample index to running sequnece
          idx=torch.cat((idx, idx_next), dim=1)#(B,T+1)
        return idx

# The training loop is incomplete in the user's provided code, so it's not included in this fix.
# You would need to instantiate the model and optimizer and then run the training loop.

In [ ]:
model = BigramLanguageModel()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if iter % eval_interval == 0 or iter == max_iters - 1:
        print(f"step {iter}: loss {loss.item()}")

step 0: loss 4.193199634552002
step 500: loss 2.6422359943389893
step 1000: loss 2.5968575477600098
step 1500: loss 2.3634629249572754
step 2000: loss 2.318666696548462
step 2500: loss 2.281993865966797
step 3000: loss 2.3047988414764404
step 3500: loss 2.4023983478546143
step 4000: loss 2.3705263137817383
step 4500: loss 2.3104121685028076
step 4999: loss 2.163257122039795


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)
# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3 # Corrected hyphen to minus sign
dropout = 0.1 # Added dropout definition

eval_iters = 200
n_embd = 32

class Head(nn.Module):
  """Single Head of Self Attention"""
  def __init__(self, head_size):
    super().__init__()
    self.key=nn.Linear(n_embd, head_size, bias=False)
    self.query=nn.Linear(n_embd, head_size, bias=False)
    self.value=nn.Linear(n_embd, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
  def forward(self,x):
    B,T,C=x.shape
    k=self.key(x)
    q=self.query(x)
    #Compute attention score(""affinities)
    wei =q @ k.transpose(-2,-1)* C**-0.5
    wei= wei.masked_fill(self.tril[:T,:T]==0, float('-inf')) # Corrected trill to tril
    wei=F.softmax(wei, dim=-1)
    #perfrom the weighted aggregation of values
    v=self.value(x)# (B,T,C) # Corrected values to value
    out =wei @ v
    return out

class FeedForward(nn.Module):
  """A simple linear layer followed by a non-linearity"""
  def __init__(self, n_embd):
    super().__init__()
    self.net=nn.Sequential(
        nn.Linear(n_embd, 4*n_embd),
        nn.ReLU(),
        nn.Linear(4*n_embd, n_embd),
        nn.Dropout(dropout),
      )
  def forward(self, x):
     return self.net(x)




class MultiHeadAttention(nn.Module):
  """Multiple heads of self attention model----By rrunning multiple heads in parrallel"""
  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads=nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj=nn.Linear(n_embd, n_embd)
    self.dropout=nn.Dropout(dropout)
  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)
    out =self.proj(out)
    out=self.dropout(out) # Apply dropout and return the result
    return out

class BatchNorm1d:
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps=eps
    #parameters (trained with backdrop)
    self.gamma=torch.ones(dim)
    self.beta=torch.zeros(dim)
  def __call__(self,x):
    #cal forward pass
    xmean= x.mean(1, keepdim=True)#batch mean
    xvar = x.var(1, keepdim=True) #batch variance
    xhat=(x-xmean)/torch.sqrt(xvar + self.eps) # normalize to unit varaince
    self.out =self.gamma * xhat + self.beta
    return self.out
  def parameters(self):
    return [self.gamma, self.beta]

class Block(nn.Module):
  """Transformer block: communication followed by computation"""
  def __init__(self, n_embd, n_head):
    # n_embd: embedding dimension, n_head: the number of heads we'd like
    super().__init__()
    head_size=n_embd//n_head
    self.sa=MultiHeadAttention(n_head, head_size)
    self.ffwd=FeedForward(n_embd)
    self.ln1=nn.LayerNorm(n_embd)
    self.ln2=nn.LayerNorm(n_embd)
  def forward(self, x):
    x=x+self.sa(self.ln1(x))
    x=x+self.ffwd(self.ln2(x))
    return x



class BigramLanguageModel(nn.Module): # Corrected indentation
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd) # Added positional embedding
        self.Block=nn.Sequential(
            Block(n_embd, n_head=4),
            Block(n_embd, n_head=4),
            Block(n_embd, n_head=4),
            nn.LayerNorm(n_embd)
        )
        self.lm_head=nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape # Get B and T from idx
        #positional encoding
        tok_embd = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device)) # (T,C), use idx.device
        x = tok_embd + pos_emb # (B,T,C)
        x=self.Block(x)#Applied one self attention head
        logits = self.lm_head(x) # (B,T,vocab_size)
        if targets is None:
            loss = None
        else:
          B, T, C=logits.shape
          logits=logits.view(B*T,C)
          targets=targets.view(B*T)
          loss=F.cross_entropy(logits, targets)
        return logits, loss
    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in current context
        for _ in range(max_new_tokens):
          # crop idsx to the last bloc_size tokens
          idx_cond=idx[:, -block_size:]
          #do the prediciton
          logits, loss=self(idx_cond) # Use idx_cond for prediction
          #focus only on last time step
          logits=logits[:,-1,:]
          #apply softmax to do prediction
          probs=F.softmax(logits, dim=-1)# (B,C)
          #sample from distribution
          idx_next=torch.multinomial(probs, num_samples=1)#(B,1)
          #append sample index to running sequnece
          idx=torch.cat((idx, idx_next), dim=1)#(B,T+1)
        return idx

# The training loop is incomplete in the user's provided code, so it's not included in this fix.
# You would need to instantiate the model and optimizer and then run the training loop.

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

model = BigramLanguageModel()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # evaluate loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.3103, val loss 4.3097
step 500: train loss 2.4110, val loss 2.4123
step 1000: train loss 2.2752, val loss 2.2859
step 1500: train loss 2.1725, val loss 2.2143
step 2000: train loss 2.1424, val loss 2.1723
step 2500: train loss 2.0927, val loss 2.1559
step 3000: train loss 2.0822, val loss 2.1332
step 3500: train loss 2.0630, val loss 2.1169
step 4000: train loss 2.0397, val loss 2.0850
step 4500: train loss 2.0263, val loss 2.0915
step 4999: train loss 1.9961, val loss 2.0732


In [ ]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))
#open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))

NameError: name 'm' is not defined

LAYERNORM

In [ ]:
class BatchNorm1d:
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps=eps
    #parameters (trained with backdrop)
    self.gamma=torch.ones(dim)
    self.beta=torch.zeros(dim)
  def __call__(self,x):
    #cal forward pass
    xmean= x.mean(1, keepdim=True)#batch mean
    xvar = x.var(1, keepdim=True) #batch variance
    xhat=(x-xmean)/torch.sqrt(xvar + self.eps) # normalize to unit varaince
    self.out =self.gamma * xhat + self.beta
    return self.out
  def parameters(self):
    return [self.gamma, self.beta]
torch.manual_seed(1337)
module = BatchNorm1d(100)
x=torch.randn(32,100)
x=module(x)
x.shape

torch.Size([32, 100])

In [31]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

class Head(nn.Module):
  """Single Head of Self Attention"""
  def __init__(self, head_size):
    super().__init__()
    self.key=nn.Linear(n_embd, head_size, bias=False)
    self.query=nn.Linear(n_embd, head_size, bias=False)
    self.value=nn.Linear(n_embd, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
    self.dropout=nn.Dropout(dropout)
  def forward(self,x):
    B,T,C=x.shape
    k=self.key(x)
    q=self.query(x)
    #Compute attention score(""affinities)
    wei =q @ k.transpose(-2,-1)* C**-0.5
    wei= wei.masked_fill(self.tril[:T,:T]==0, float('-inf')) # Corrected trill to tril
    wei=F.softmax(wei, dim=-1)
    #perfrom the weighted aggregation of values
    v=self.value(x)# (B,T,C) # Corrected values to value
    out =wei @ v
    return out

class FeedForward(nn.Module):
  """A simple linear layer followed by a non-linearity"""
  def __init__(self, n_embd):
    super().__init__()
    self.net=nn.Sequential(
        nn.Linear(n_embd, 4*n_embd),
        nn.ReLU(),
        nn.Linear(4*n_embd, n_embd),
        nn.Dropout(dropout),
      )
  def forward(self, x):
     return self.net(x)




class MultiHeadAttention(nn.Module):
  """Multiple heads of self attention model----By rrunning multiple heads in parrallel"""
  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads=nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj=nn.Linear(head_size * num_heads, n_embd)
    self.dropout=nn.Dropout(dropout)
  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)
    out =self.proj(out)
    out=self.dropout(out) # Apply dropout and return the result
    return out

class BatchNorm1d:
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps=eps
    #parameters (trained with backdrop)
    self.gamma=torch.ones(dim)
    self.beta=torch.zeros(dim)
  def __call__(self,x):
    #cal forward pass
    xmean= x.mean(1, keepdim=True)#batch mean
    xvar = x.var(1, keepdim=True) #batch variance
    xhat=(x-xmean)/torch.sqrt(xvar + self.eps) # normalize to unit varaince
    self.out =self.gamma * xhat + self.beta
    return self.out
  def parameters(self):
    return [self.gamma, self.beta]

class Block(nn.Module):
  """Transformer block: communication followed by computation"""
  def __init__(self, n_embd, n_head):
    # n_embd: embedding dimension, n_head: the number of heads we'd like
    super().__init__()
    head_size=n_embd//n_head
    self.sa=MultiHeadAttention(n_head, head_size)
    self.ffwd=FeedForward(n_embd)
    self.ln1=nn.LayerNorm(n_embd)
    self.ln2=nn.LayerNorm(n_embd)
  def forward(self, x):
    x=x+self.sa(self.ln1(x))
    x=x+self.ffwd(self.ln2(x))
    return x



class BigramLanguageModel(nn.Module): # Corrected indentation
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd) # Added positional embedding
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head=nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape # Get B and T from idx
        #positional encoding
        tok_embd = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device)) # (T,C), use idx.device
        x = tok_embd + pos_emb # (B,T,C)
        x=self.blocks(x)#Applied one self attention head
        x = self.ln_f(x)
        logits = self.lm_head(x) # (B,T,vocab_size)
        if targets is None:
            loss = None
        else:
          B, T, C=logits.shape
          logits=logits.view(B*T,C)
          targets=targets.view(B*T)
          loss=F.cross_entropy(logits, targets)
        return logits, loss
    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in current context
        for _ in range(max_new_tokens):
          # crop idsx to the last bloc_size tokens
          idx_cond=idx[:, -block_size:]
          #do the prediciton
          logits, loss=self(idx_cond) # Use idx_cond for prediction
          #focus only on last time step
          logits=logits[:,-1,:]
          #apply softmax to do prediction
          probs=F.softmax(logits, dim=-1)# (B,C)
          #sample from distribution
          idx_next=torch.multinomial(probs, num_samples=1)#(B,1)
          #append sample index to running sequnece
          idx=torch.cat((idx, idx_next), dim=1)#(B,T+1)
        return idx

# The training loop is incomplete in the user's provided code, so it's not included in this fix.
# You would need to instantiate the model and optimizer and then run the training loop.

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

model = BigramLanguageModel()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # evaluate loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.2849, val loss 4.2823
